In [1]:
import pandas as pd

ball_by_ball_data = pd.read_csv("deliveries.csv")
matches_data = pd.read_csv("matches.csv")

C:\Users\Pranav Koka\AppData\Local\Temp\ipykernel_952\2815139961.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
matches_data['match_id'] = matches_data['id']
matches_data.drop('id', axis = 1, inplace=True)

In [3]:
data = pd.merge(ball_by_ball_data, matches_data, on='match_id', how='left')
data

,match_id,inning,batting_team,bowling_team,over,ball,batter,bowler,non_striker,batsman_runs,...,toss_decision,winner,result,result_margin,target_runs,target_overs,super_over,method,umpire1,umpire2
0,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,1,SC Ganguly,P Kumar,BB McCullum,0,...,field,Kolkata Knight Riders,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen
1,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,2,BB McCullum,P Kumar,SC Ganguly,0,...,field,Kolkata Knight Riders,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen
2,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,3,BB McCullum,P Kumar,SC Ganguly,0,...,field,Kolkata Knight Riders,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen
3,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,4,BB McCullum,P Kumar,SC Ganguly,0,...,field,Kolkata Knight Riders,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen
4,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,5,BB McCullum,P Kumar,SC Ganguly,0,...,field,Kolkata Knight Riders,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243812,1370353,2,Chennai Super Kings,Gujarat Titans,14,2,S Dube,MM Sharma,RA Jadeja,1,...,field,Chennai Super Kings,wickets,5.0,171.0,15.0,N,D/L,Nitin Menon,RJ Tucker
243813,1370353,2,Chennai Super Kings,Gujarat Titans,14,3,RA Jadeja,MM Sharma,S Dube,1,...,field,Chennai Super Kings,wickets,5.0,171.0,15.0,N,D/L,Nitin Menon,RJ Tucker
243814,1370353,2,Chennai Super Kings,Gujarat Titans,14,4,S Dube,MM Sharma,RA Jadeja,1,...,field,Chennai Super Kings,wickets,5.0,171.0,15.0,N,D/L,Nitin Menon,RJ Tucker
243815,1370353,2,Chennai Super Kings,Gujarat Titans,14,5,RA Jadeja,MM Sharma,S Dube,6,...,field,Chennai Super Kings,wickets,5.0,171.0,15.0,N,D/L,Nitin Menon,RJ Tucker


In [4]:
data.columns

Index(['match_id', 'inning', 'batting_team', 'bowling_team', 'over', 'ball',
       'batter', 'bowler', 'non_striker', 'batsman_runs', 'extra_runs',
       'total_runs', 'extras_type', 'is_wicket', 'player_dismissed',
       'dismissal_kind', 'fielder', 'season', 'city', 'date', 'match_type',
       'player_of_match', 'venue', 'team1', 'team2', 'toss_winner',
       'toss_decision', 'winner', 'result', 'result_margin', 'target_runs',
       'target_overs', 'super_over', 'method', 'umpire1', 'umpire2'],
      dtype='object')

In [5]:
#ensuring that wides, noballs, and penalties don't count as a ball
for i in range(len(data)):
    if data.extras_type[i] in ['wides', 'noballs', 'penalty']:
        k = True
        j = i+1
        # Ensure j does not exceed DataFrame length
        while k and j < len(data):
            if data.ball[j] > data.ball[i]:
                data.loc[j, 'ball'] = data.ball[j] - 1
                j += 1
            else:
                k = False
        data.loc[i, 'ball'] = data.ball[i] - 1


In [6]:
#get the cumulative innings total runs
innings_total_list = []
i = 0
x = True
while i < len(data):
    innings_total = data.total_runs[i]
    innings_total_list.append(innings_total)
    i = i+1
    while i < len(data) and data.match_id[i] == data.match_id[i-1] and data.inning[i] == data.inning[i-1]:
        innings_total = innings_total + data.total_runs[i]
        innings_total_list.append(innings_total)
        i = i+1

In [7]:
data['cumulative_innings_total'] = innings_total_list 
data['cumulative_balls'] = data['over']*6 + data['ball']

In [8]:
#get the total runs scored in that innings
data['innings_total_runs'] = data.groupby(['match_id', 'inning'])['cumulative_innings_total'].transform('max')

#get the total balls bowled in that innings
data['innings_total_balls'] = data.groupby(['match_id', 'inning'])['cumulative_balls'].transform('max')

In [9]:
data['match_total_runs'] = data['target_runs'] - 1 + data['target_runs'] - 1 - data['result_margin']

In [10]:
#get the first row in each innings and then add the total balls in each innings to get the total balls bowled in the match
data_first_rows = data.groupby(['match_id', 'inning']).first().reset_index()
data_first_rows['match_total_balls'] = data_first_rows.groupby('match_id')['innings_total_balls'].transform('sum')
to_merge_df = data_first_rows[['match_id', 'match_total_balls']].drop_duplicates()

In [11]:
data = pd.merge(data, to_merge_df, on='match_id', how='left')

In [12]:
data[0:10]

,match_id,inning,batting_team,bowling_team,over,ball,batter,bowler,non_striker,batsman_runs,...,super_over,method,umpire1,umpire2,cumulative_innings_total,cumulative_balls,innings_total_runs,innings_total_balls,match_total_runs,match_total_balls
0,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,1,SC Ganguly,P Kumar,BB McCullum,0,...,N,NaN,Asad Rauf,RE Koertzen,1,1,222,120,304.0,211
1,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,2,BB McCullum,P Kumar,SC Ganguly,0,...,N,NaN,Asad Rauf,RE Koertzen,1,2,222,120,304.0,211
2,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,2,BB McCullum,P Kumar,SC Ganguly,0,...,N,NaN,Asad Rauf,RE Koertzen,2,2,222,120,304.0,211
3,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,3,BB McCullum,P Kumar,SC Ganguly,0,...,N,NaN,Asad Rauf,RE Koertzen,2,3,222,120,304.0,211
4,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,4,BB McCullum,P Kumar,SC Ganguly,0,...,N,NaN,Asad Rauf,RE Koertzen,2,4,222,120,304.0,211
5,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,5,BB McCullum,P Kumar,SC Ganguly,0,...,N,NaN,Asad Rauf,RE Koertzen,2,5,222,120,304.0,211
6,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,6,BB McCullum,P Kumar,SC Ganguly,0,...,N,NaN,Asad Rauf,RE Koertzen,3,6,222,120,304.0,211
7,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,1,1,BB McCullum,Z Khan,SC Ganguly,0,...,N,NaN,Asad Rauf,RE Koertzen,3,7,222,120,304.0,211
8,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,1,2,BB McCullum,Z Khan,SC Ganguly,4,...,N,NaN,Asad Rauf,RE Koertzen,7,8,222,120,304.0,211
9,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,1,3,BB McCullum,Z Khan,SC Ganguly,4,...,N,NaN,Asad Rauf,RE Koertzen,11,9,222,120,304.0,211


In [13]:
data['batter_total_runs'] = data.groupby(['match_id','batter'])['batsman_runs'].cumsum()
data['batter_total_balls'] = data[(data['extras_type'] != 'wides') & (data['extras_type'] != 'penalty')].groupby(['match_id','batter']).cumcount() + 1
#will forward fill the last valid value to the nan value
data['batter_total_balls'] = data.groupby(['match_id', 'batter'])['batter_total_balls'].transform(lambda x: x.ffill())
#if the first ball that the batter faced was a wide then there is nothing to ffill so we fill these with 0
data['batter_total_balls'] = data['batter_total_balls'].fillna(0)

In [14]:
data['bowler_total_runs'] = data[(data['extras_type'] != 'byes') & (data['extras_type'] != 'legbyes') & (data['extras_type'] != 'penalty')].groupby(['match_id', 'bowler'])['total_runs'].cumsum()
data['bowler_total_balls'] = data[(data['extras_type'] != 'wides') & (data['extras_type'] != 'noballs') & (data['extras_type'] != 'penalty')].groupby(['match_id','bowler']).cumcount() + 1
#will forward fill the last valid value to the nan value
data['bowler_total_runs'] = data.groupby(['match_id', 'bowler'])['bowler_total_runs'].transform(lambda x: x.ffill())
#if the first ball bowled by a bowler is a bye then there is nothing to ffill so we fill these with 0
data['bowler_total_runs'] = data['bowler_total_runs'].fillna(0)
#will forward fill the last valid value to the nan value
data['bowler_total_balls'] = data.groupby(['match_id', 'bowler'])['bowler_total_balls'].transform(lambda x: x.ffill())
#if the first ball bowled by a bowler is a noball then there is nothing to ffill so we fill these with 0
data['bowler_total_balls'] = data['bowler_total_balls'].fillna(0)

In [15]:
data['bowler_economy'] = data['bowler_total_runs']/data['bowler_total_balls'] * 6

In [16]:
data['batter_SR'] = data['batter_total_runs']/data['batter_total_balls'] * 100

In [17]:
data.columns

Index(['match_id', 'inning', 'batting_team', 'bowling_team', 'over', 'ball',
       'batter', 'bowler', 'non_striker', 'batsman_runs', 'extra_runs',
       'total_runs', 'extras_type', 'is_wicket', 'player_dismissed',
       'dismissal_kind', 'fielder', 'season', 'city', 'date', 'match_type',
       'player_of_match', 'venue', 'team1', 'team2', 'toss_winner',
       'toss_decision', 'winner', 'result', 'result_margin', 'target_runs',
       'target_overs', 'super_over', 'method', 'umpire1', 'umpire2',
       'cumulative_innings_total', 'cumulative_balls', 'innings_total_runs',
       'innings_total_balls', 'match_total_runs', 'match_total_balls',
       'batter_total_runs', 'batter_total_balls', 'bowler_total_runs',
       'bowler_total_balls', 'bowler_economy', 'batter_SR'],
      dtype='object')

In [18]:
data.season.unique()

array(['2007/08', '2009', '2009/10', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', '2019', '2020/21', '2021', '2022',
       '2023'], dtype=object)

In [19]:
data_2008 = data[data['season'] =='2007/08']
data_2009 = data[data['season'] =='2009']
data_2010 = data[data['season'] =='2009/10']
data_2011 = data[data['season'] =='2011']
data_2012 = data[data['season'] =='2012']
data_2013 = data[data['season'] =='2013']
data_2014 = data[data['season'] =='2014']
data_2015 = data[data['season'] =='2015']
data_2016 = data[data['season'] =='2016']
data_2017 = data[data['season'] =='2017']
data_2018 = data[data['season'] =='2018']
data_2019 = data[data['season'] =='2019']
data_2020 = data[data['season'] =='2020/21']
data_2021 = data[data['season'] =='2021']
data_2022 = data[data['season'] =='2022']
data_2023 = data[data['season'] =='2023']

In [20]:
xR_data = data

In [21]:
#xR_data.to_csv("xR_data.csv", index=False)

In [22]:
unnecessary_cols = [
    'batting_team', 'batsman_runs', 'over', 'ball', 'total_runs',
    'non_striker', 'extra_runs', 'extras_type', 'is_wicket',
    'player_dismissed', 'fielder', 'season', 'city',
    'player_of_match', 'venue', 'team1', 'team2', 'toss_winner',
    'toss_decision', 'winner', 'result', 'result_margin', 'target_runs',
    'target_overs', 'super_over', 'method', 'umpire1', 'umpire2',
    'cumulative_innings_total', 'innings_total_runs',
    'innings_total_balls', 'match_total_runs', 'match_total_balls',
    'bowler_total_runs', 'bowler_total_balls', 'bowler_economy',
]

In [23]:
# Drop the unnecessary columns
df_reduced = data.drop(columns=unnecessary_cols)

# Display the new DataFrame
df_reduced

,match_id,inning,bowling_team,batter,bowler,dismissal_kind,date,match_type,cumulative_balls,batter_total_runs,batter_total_balls,batter_SR
0,335982,1,Royal Challengers Bangalore,SC Ganguly,P Kumar,NaN,2008-04-18,League,1,0,1.0,0.000000
1,335982,1,Royal Challengers Bangalore,BB McCullum,P Kumar,NaN,2008-04-18,League,2,0,1.0,0.000000
2,335982,1,Royal Challengers Bangalore,BB McCullum,P Kumar,NaN,2008-04-18,League,2,0,1.0,0.000000
3,335982,1,Royal Challengers Bangalore,BB McCullum,P Kumar,NaN,2008-04-18,League,3,0,2.0,0.000000
4,335982,1,Royal Challengers Bangalore,BB McCullum,P Kumar,NaN,2008-04-18,League,4,0,3.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
243812,1370353,2,Gujarat Titans,S Dube,MM Sharma,NaN,2023-05-29,Final,86,31,20.0,155.000000
243813,1370353,2,Gujarat Titans,RA Jadeja,MM Sharma,NaN,2023-05-29,Final,87,5,4.0,125.000000
243814,1370353,2,Gujarat Titans,S Dube,MM Sharma,NaN,2023-05-29,Final,88,32,21.0,152.380952
243815,1370353,2,Gujarat Titans,RA Jadeja,MM Sharma,NaN,2023-05-29,Final,89,11,5.0,220.000000


In [24]:
# ball_by_ball_data[0:10]
df_reduced['batter_score'] = df_reduced.groupby(['match_id', 'batter'])['batter_total_runs'].transform('max')
df_reduced

,match_id,inning,bowling_team,batter,bowler,dismissal_kind,date,match_type,cumulative_balls,batter_total_runs,batter_total_balls,batter_SR,batter_score
0,335982,1,Royal Challengers Bangalore,SC Ganguly,P Kumar,NaN,2008-04-18,League,1,0,1.0,0.000000,10
1,335982,1,Royal Challengers Bangalore,BB McCullum,P Kumar,NaN,2008-04-18,League,2,0,1.0,0.000000,158
2,335982,1,Royal Challengers Bangalore,BB McCullum,P Kumar,NaN,2008-04-18,League,2,0,1.0,0.000000,158
3,335982,1,Royal Challengers Bangalore,BB McCullum,P Kumar,NaN,2008-04-18,League,3,0,2.0,0.000000,158
4,335982,1,Royal Challengers Bangalore,BB McCullum,P Kumar,NaN,2008-04-18,League,4,0,3.0,0.000000,158
...,...,...,...,...,...,...,...,...,...,...,...,...,...
243812,1370353,2,Gujarat Titans,S Dube,MM Sharma,NaN,2023-05-29,Final,86,31,20.0,155.000000,32
243813,1370353,2,Gujarat Titans,RA Jadeja,MM Sharma,NaN,2023-05-29,Final,87,5,4.0,125.000000,15
243814,1370353,2,Gujarat Titans,S Dube,MM Sharma,NaN,2023-05-29,Final,88,32,21.0,152.380952,32
243815,1370353,2,Gujarat Titans,RA Jadeja,MM Sharma,NaN,2023-05-29,Final,89,11,5.0,220.000000,15


In [25]:
dismissal_kinds = df_reduced['dismissal_kind'].unique()
dismissal_kinds

array([nan, 'caught', 'bowled', 'run out', 'lbw', 'retired hurt',
       'stumped', 'caught and bowled', 'hit wicket',
       'obstructing the field', 'retired out'], dtype=object)

In [26]:
import pandas as pd

# Read the player.csv file into a DataFrame
players = pd.read_csv("player.csv")

# Display the DataFrame
players

new_players = pd.DataFrame()
new_players[['Player_Name', 'Bowling_Skill']] = players[['Player_Name', 'Bowling_Skill']]

new_players.dropna(inplace=True)

new_players

df_reduced['Bowling_Skill'] = df_reduced['bowler'].map(new_players.set_index('Player_Name')['Bowling_Skill'])
df_reduced['Bowling_Skill'] = df_reduced['Bowling_Skill'].fillna('nan')
df_reduced[20:40]

df_reduced['Bowling_Skill'] = df_reduced['Bowling_Skill'].replace({
  'Right-arm medium': 'Right-arm pace',
  'Right-arm fast-medium': 'Right-arm pace',
  'Right-arm medium-fast': 'Right-arm pace',
  'Right-arm fast': 'Right-arm pace',
  'Left-arm medium': 'Left-arm pace',
  'Left-arm fast-medium': 'Left-arm pace',
  'Left-arm medium-fast': 'Left-arm pace',
  'Left-arm fast': 'Left-arm pace',
  'Right-arm offbreak': 'Right-arm offspin',
  'Legbreak googly': 'Right-arm legspin',
  'Legbreak': 'Right-arm legspin',
  'Slow left-arm orthodox': 'Left-arm offspin',
  'Slow left-arm chinaman': 'Left-arm legspin',
})

unique_bowling_types = df_reduced['Bowling_Skill'].unique()
unique_bowling_types

array(['Right-arm pace', 'Left-arm pace', 'Left-arm offspin',
       'Right-arm legspin', 'Right-arm offspin', 'Left-arm legspin',
       'nan'], dtype=object)

In [27]:
print(df_reduced.columns)

Index(['match_id', 'inning', 'bowling_team', 'batter', 'bowler',
       'dismissal_kind', 'date', 'match_type', 'cumulative_balls',
       'batter_total_runs', 'batter_total_balls', 'batter_SR', 'batter_score',
       'Bowling_Skill'],
      dtype='object')


In [28]:
# player name, runs scored in match, no. of each bowler type, for each batter: match-wise rows, batting pos, innings, venue

In [29]:
df_reduced = df_reduced.loc[0:140]

In [30]:
df_reduced['no_of_right_arm_pacers_faced'] = 0
df_reduced['no_of_left_arm_pacers_faced'] = 0
df_reduced['no_of_right_arm_offspinners_faced'] = 0
df_reduced['no_of_left_arm_offpacers_faced'] = 0
df_reduced['no_of_right_arm_legspinners_faced'] = 0
df_reduced['no_of_left_arm_legspinners_faced'] = 0


# Dictionary to keep track of bowlers faced by each batter in each match
bowlers_faced = {}

# Iterate through the DataFrame
for idx, row in df_reduced.iterrows():
    match_id = row['match_id']
    batter = row['batter']
    bowler = row['bowler']
    skill = row['Bowling_Skill']
    
    if match_id not in bowlers_faced:
        bowlers_faced[match_id] = {}
    
    if batter not in bowlers_faced[match_id]:
        bowlers_faced[match_id][batter] = {'Right-arm pace': set(), 'Left-arm pace': set(), 'Right-arm offspin': set(), 'Left-arm offspin': set(), 'Right-arm legspin': set(), 'Left-arm legspin': set(), 'nan': set()}
    
    #if bowler not in bowlers_faced[match_id][batter][skill]:
    bowlers_faced[match_id][batter][skill].add(bowler)
    
    df_reduced.loc[df_reduced.index == idx, 'no_of_right_arm_pacers_faced'] = len(bowlers_faced[match_id][batter]['Right-arm pace'])
    df_reduced.loc[df_reduced.index == idx, 'no_of_left_arm_pacers_faced'] = len(bowlers_faced[match_id][batter]['Left-arm pace'])
    df_reduced.loc[df_reduced.index == idx, 'no_of_right_arm_offspinners_faced'] = len(bowlers_faced[match_id][batter]['Right-arm offspin'])
    df_reduced.loc[df_reduced.index == idx, 'no_of_left_arm_offspinners_faced'] = len(bowlers_faced[match_id][batter]['Left-arm offspin'])
    df_reduced.loc[df_reduced.index == idx, 'no_of_right_arm_legspinners_faced'] = len(bowlers_faced[match_id][batter]['Right-arm legspin'])
    df_reduced.loc[df_reduced.index == idx, 'no_of_left_arm_legspinners_faced'] = len(bowlers_faced[match_id][batter]['Left-arm legspin'])
    df_reduced.loc[df_reduced.index == idx, 'no_of_nans'] = len(bowlers_faced[match_id][batter]['nan'])

    # if skill == 'Right-arm pace':
    #     df_reduced.loc[df_reduced.index == idx, 'no_of_right_arm_pacers_faced'] = len(bowlers_faced[match_id][batter]['Right-arm pace'])
    # if skill == 'Left-arm pace':
    #     df_reduced.loc[df_reduced.index == idx, 'no_of_left_arm_pacers_faced'] = len(bowlers_faced[match_id][batter]['Left-arm pace'])
    # if skill == 'Right-arm offspin':
    #     df_reduced.loc[df_reduced.index == idx, 'no_of_right_arm_offspinners_faced'] = len(bowlers_faced[match_id][batter]['Right-arm offspin'])
    # if skill == 'Left-arm offspin':
    #     df_reduced.loc[df_reduced.index == idx, 'no_of_left_arm_offspinners_faced'] = len(bowlers_faced[match_id][batter]['Left-arm offspin'])
    # if skill == 'Right-arm legspin':
    #     df_reduced.loc[df_reduced.index == idx, 'no_of_right_arm_legspinners_faced'] = len(bowlers_faced[match_id][batter]['Right-arm legspin'])
    # if skill == 'Left-arm legspin':
    #     df_reduced.loc[df_reduced.index == idx, 'no_of_left_arm_legspinners_faced'] = len(bowlers_faced[match_id][batter]['Left-arm legspin'])
    # if skill == 'nan':
    #     df_reduced.loc[df_reduced.index == idx, 'no_of_nans'] = len(bowlers_faced[match_id][batter]['nan'])


C:\Users\Pranav Koka\AppData\Local\Temp\ipykernel_952\971501663.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reduced['no_of_right_arm_pacers_faced'] = 0
C:\Users\Pranav Koka\AppData\Local\Temp\ipykernel_952\971501663.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reduced['no_of_left_arm_pacers_faced'] = 0
C:\Users\Pranav Koka\AppData\Local\Temp\ipykernel_952\971501663.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [31]:
bowlers_faced

{335982: {'SC Ganguly': {'Right-arm pace': {'AA Noffke', 'P Kumar'},
   'Left-arm pace': {'Z Khan'},
   'Right-arm offspin': set(),
   'Left-arm offspin': set(),
   'Right-arm legspin': set(),
   'Left-arm legspin': set(),
   'nan': set()},
  'BB McCullum': {'Right-arm pace': {'AA Noffke', 'JH Kallis', 'P Kumar'},
   'Left-arm pace': {'Z Khan'},
   'Right-arm offspin': set(),
   'Left-arm offspin': {'SB Joshi'},
   'Right-arm legspin': {'CL White'},
   'Left-arm legspin': set(),
   'nan': set()},
  'RT Ponting': {'Right-arm pace': {'AA Noffke', 'JH Kallis'},
   'Left-arm pace': {'Z Khan'},
   'Right-arm offspin': set(),
   'Left-arm offspin': {'SB Joshi'},
   'Right-arm legspin': set(),
   'Left-arm legspin': set(),
   'nan': set()},
  'DJ Hussey': {'Right-arm pace': {'AA Noffke'},
   'Left-arm pace': {'Z Khan'},
   'Right-arm offspin': set(),
   'Left-arm offspin': {'SB Joshi'},
   'Right-arm legspin': {'CL White'},
   'Left-arm legspin': set(),
   'nan': set()},
  'Mohammad Hafeez': 

In [32]:
df_reduced[120:140]

,match_id,inning,bowling_team,batter,bowler,dismissal_kind,date,match_type,cumulative_balls,batter_total_runs,...,batter_score,Bowling_Skill,no_of_right_arm_pacers_faced,no_of_left_arm_pacers_faced,no_of_right_arm_offspinners_faced,no_of_left_arm_offpacers_faced,no_of_right_arm_legspinners_faced,no_of_left_arm_legspinners_faced,no_of_left_arm_offspinners_faced,no_of_nans
120,335982,1,Royal Challengers Bangalore,BB McCullum,P Kumar,NaN,2008-04-18,League,117,150,...,158,Right-arm pace,3,1,0,0,1,0,1.0,0.0
121,335982,1,Royal Challengers Bangalore,BB McCullum,P Kumar,NaN,2008-04-18,League,118,150,...,158,Right-arm pace,3,1,0,0,1,0,1.0,0.0
122,335982,1,Royal Challengers Bangalore,BB McCullum,P Kumar,NaN,2008-04-18,League,119,152,...,158,Right-arm pace,3,1,0,0,1,0,1.0,0.0
123,335982,1,Royal Challengers Bangalore,BB McCullum,P Kumar,NaN,2008-04-18,League,120,158,...,158,Right-arm pace,3,1,0,0,1,0,1.0,0.0
124,335982,2,Kolkata Knight Riders,R Dravid,AB Dinda,NaN,2008-04-18,League,1,1,...,2,Right-arm pace,1,0,0,0,0,0,0.0,0.0
125,335982,2,Kolkata Knight Riders,W Jaffer,AB Dinda,NaN,2008-04-18,League,1,0,...,6,Right-arm pace,1,0,0,0,0,0,0.0,0.0
126,335982,2,Kolkata Knight Riders,W Jaffer,AB Dinda,NaN,2008-04-18,League,2,0,...,6,Right-arm pace,1,0,0,0,0,0,0.0,0.0
127,335982,2,Kolkata Knight Riders,W Jaffer,AB Dinda,NaN,2008-04-18,League,3,1,...,6,Right-arm pace,1,0,0,0,0,0,0.0,0.0
128,335982,2,Kolkata Knight Riders,R Dravid,AB Dinda,NaN,2008-04-18,League,4,2,...,2,Right-arm pace,1,0,0,0,0,0,0.0,0.0
129,335982,2,Kolkata Knight Riders,W Jaffer,AB Dinda,NaN,2008-04-18,League,5,1,...,6,Right-arm pace,1,0,0,0,0,0,0.0,0.0
